# Saving and Serializing Models with TensorFlow Keras
Source: https://www.tensorflow.org/alpha/guide/keras/saving_and_serializing#weights-only_saving

Part 1 : Saving and Serialization for Sequential Models and Functional API.


Part 2 : Saving for Custom subclass of models. The API is slightly different to Sequential or Functional.

In [1]:
import tensorflow as tf

tf.keras.backend.clear_session()  # For easy reset of notebook state.

# Part I: Saving Sequential models or Functional models


Let's consider the following model:



In [2]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')
model.summary()

Model: "3_layer_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


#### Train the Model

In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1,
                   verbose = 2)

60000/60000 - 7s - loss: 0.1413


In [4]:
# Save predictions for future checks
predictions = model.predict(x_test)

## Whole-model saving

This file includes:

- The model's architecture
- The model's weight values (which were learned during training)
- The model's training config (what you passed to compile), if any
- The optimizer and its state, if any (this enables you to restart training where you left off)


In [5]:
# Save the model
model.save('path_to_my_model.h5')

# Recreate the exact same model purely from the file
new_model = keras.models.load_model('path_to_my_model.h5')

In [7]:
import numpy as np

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, atol=1e-6)

# Note that the optimizer state is preserved as well:
# you can resume training where you left off.

## Export to SavedModel

SavedModel is a standalone serialization format for Tensorflow objects, supported by TensorFlow serving as well as TensorFlow implementations other than Python.



In [8]:
# Export the model to a SavedModel
keras.experimental.export_saved_model(model, 'path_to_saved_model')

# Recreate the exact same model
new_model = keras.experimental.load_from_saved_model('path_to_saved_model')

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, atol=1e-6)

# Note that the optimizer state is preserved as well:
# you can resume training where you left off.

W0322 05:26:37.342111 19832 deprecation.py:323] From c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\saved_model\signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0322 05:26:37.354104 19832 tf_logging.py:161] Export includes no default signature!
W0322 05:26:39.856304 19832 tf_logging.py:161] Export includes no default signature!


AssertionError: 
Not equal to tolerance rtol=1e-07, atol=1e-06

Mismatch: 79.8%
Max absolute difference: 0.97171205
Max relative difference: 764597.3
 x: array([[1.226035e-05, 1.667942e-08, 8.437489e-05, ..., 9.975024e-01,
        1.408940e-05, 9.716916e-04],
       [6.980557e-05, 1.912783e-04, 9.979590e-01, ..., 2.244759e-06,...
 y: array([[2.332846e-05, 3.303864e-08, 1.091335e-04, ..., 9.975899e-01,
        3.049750e-06, 7.981198e-04],
       [1.767552e-07, 2.149919e-04, 9.997712e-01, ..., 1.167043e-07,...

The SavedModel files that were created contain:

- A TensorFlow checkpoint containing the model weights.
- A SavedModel proto containing the underlying Tensorflow graph. Separate graphs are saved for prediction (serving), train, and evaluation. If the model wasn't compiled before, then only the inference graph gets exported.
- The model's architecture config, if available.


## Architecture-only saving

In this case, you can retrieve the "config" of the model via the `get_config()` method

In [9]:
config = model.get_config()
reinitialized_model = keras.Model.from_config(config)

# Note that the model state is not preserved! We only saved the architecture.
new_predictions = reinitialized_model.predict(x_test)
assert abs(np.sum(predictions - new_predictions)) > 0.

Alternatively use `to_json()` and `from_json()`, which uses a JSON string to store the config instead of a Python dict. This is useful to save the config to disk.

In [10]:
json_config = model.to_json()
reinitialized_model = keras.models.model_from_json(json_config)

## Weights-only saving

In this case, you can retrieve the weights values as a list of Numpy arrays via `get_weights()`, and set the state of the model via `set_weights`:

In [11]:
weights = model.get_weights()  # Retrieves the state of the model.
model.set_weights(weights)  # Sets the state of the model.

#### simplest, recommended way is just this:



In [12]:
model.save('path_to_my_model.h5')
del model
model = keras.models.load_model('path_to_my_model.h5')

## Weights-only saving in SavedModel format

Anything  else defaults to SavedModel.

In [13]:
model.save_weights('path_to_my_tf_savedmodel')

#### For total explicitness, the format can be explicitly passed via the save_format argument, which can take the value "tf" or "h5":



In [14]:
model.save_weights('path_to_my_tf_savedmodel', save_format='tf')

# Saving Subclassed Models

Sequential models and Functional models are datastructures that represent a DAG of layers. As such, they can be safely serialized and deserialized.

A subclassed model differs in that it's not a datastructure, it's a piece of code. The architecture of the model is defined via the body of the call method. This means that the architecture of the model cannot be safely serialized. To load a model, you'll need to have access to the code that created it (the code of the model subclass). Alternatively, you could be serializing this code as bytecode (e.g. via pickling), but that's unsafe and generally not portable.



#### Let's consider the following subclassed model, which follows the same structure as the model from the first section:



In [15]:
class ThreeLayerMLP(keras.Model):
  
  def __init__(self, name=None):
    super(ThreeLayerMLP, self).__init__(name=name)
    self.dense_1 = layers.Dense(64, activation='relu', name='dense_1')
    self.dense_2 = layers.Dense(64, activation='relu', name='dense_2')
    self.pred_layer = layers.Dense(10, activation='softmax', name='predictions')
    
  def call(self, inputs):
    x = self.dense_1(inputs)
    x = self.dense_2(x)
    return self.pred_layer(x)

def get_model():
  return ThreeLayerMLP(name='3_layer_mlp')

model = get_model()

First of all, a subclassed model that has never been used cannot be saved.

Always runt he code alteast once before saving. 

Let's train teh model,so as to give it a state.



In [16]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

60000/60000 [==============================] - 9s 142us/sample - loss: 0.3048


In [20]:
# Using save_weights
model.save_weights('path_to_my_weights', save_format='tf')

In [21]:
# Save predictions for future checks
predictions = model.predict(x_test)

# Also save the loss on the first batch
# to later assert that the optimizer state was preserved

first_batch_loss = model.train_on_batch(x_train[:64], y_train[:64])

#### To restore your model, you will need access to the code that created the model object.



In [22]:
# Recreate the model
new_model = get_model()
new_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.RMSprop())

# This initializes the variables used by the optimizers,
# as well as any stateful metric variables
new_model.train_on_batch(x_train[:1], y_train[:1])

# Load the state of the old model
new_model.load_weights('path_to_my_weights')

# Check that the model state has been preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, atol=1e-6)

# The optimizer state is preserved as well,
# so you can resume training where you left off
new_first_batch_loss = new_model.train_on_batch(x_train[:64], y_train[:64])
assert first_batch_loss == new_first_batch_loss